In [2]:
# diagnosis of lung damage based on medical imagery .ipynb Prithvi

In [2]:
# install necessary models, datasets, and libraries
!pip install monai
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install opencv-python-headless

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import os
import sys
import shutil
import tempfile
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2


import torch

from monai.apps import download_and_extract
from monai.config import print_config
from monai.metrics import ROCAUCMetric
from monai.networks.nets import DenseNet121
from monai.transforms import *
from monai.data import Dataset, DataLoader
from monai.utils import set_determinism
print_config()

MONAI version: 1.3.2
Numpy version: 2.1.1
Pytorch version: 2.0.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 59a7211070538586369afd4a01eca0a7fe2e742e
MONAI __file__: /home/<username>/.local/lib/python3.10/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: NOT INSTALLED or UNKNOWN VERSION.
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
scipy version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 10.4.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.2
tqdm version: NOT INSTALLED or UNKNOWN VERSION.
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.6
pandas version: 2.2.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or 

In [8]:
# bring in the already imported data from the data directory
train_directory='../data/ctimages/Data/train'
val_directory='../data/ctimages/Data/valid'
test_directory='../data/ctimages/Data/test'

In [9]:
# Get sorted class names from the train dataset
class_names = sorted(os.listdir(train_directory))
num_class = len(class_names)

# Create a list of all image files with their corresponding labels
image_file_list, image_label_list = [], []

for label, class_name in enumerate(class_names):
    class_dir = os.path.join(train_directory, class_name)
    image_files = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    
    image_file_list.extend(image_files)
    image_label_list.extend([label] * len(image_files))

print(class_names)

['adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib', 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa', 'normal', 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa']


In [10]:
# List and sort validation class names
v_class_names = sorted(os.listdir(val_directory))
v_num_class = len(v_class_names)

# Initialize lists for image file paths and labels
v_image_file_list, v_image_label_list = [], []

# Iterate over each class and collect image file paths and corresponding labels
for label, class_name in enumerate(v_class_names):
    class_dir = os.path.join(val_directory, class_name)
    image_files = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    
    v_image_file_list.extend(image_files)
    v_image_label_list.extend([label] * len(image_files))

print(v_class_names)

['adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib', 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa', 'normal', 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa']


In [11]:
# List and sort test class names
t_class_names = sorted(os.listdir(test_directory))
t_num_class = len(t_class_names)

# Initialize lists for image file paths and labels
t_image_file_list, t_image_label_list = [], []

# Iterate over each class and collect image file paths and corresponding labels
for label, class_name in enumerate(t_class_names):
    class_dir = os.path.join(test_directory, class_name)
    image_files = [os.path.join(class_dir, img) for img in os.listdir(class_dir)]
    
    t_image_file_list.extend(image_files)
    t_image_label_list.extend([label] * len(image_files))

print(t_class_names)

['adenocarcinoma', 'large.cell.carcinoma', 'normal', 'squamous.cell.carcinoma']


In [12]:
# prepare training, validation, and testing datasets
trainX=np.array(image_file_list)
trainY=np.array(image_label_list)
valX=np.array(v_image_file_list)
valY=np.array(v_image_label_list)
testX=np.array(t_image_file_list)
testY=np.array(t_image_label_list)

print("Training count =",len(trainX),"Validation count =", len(valX), "Test count =",len(testX))

Training count = 613 Validation count = 72 Test count = 315


In [13]:
class SumDimension(Transform):
    def __init__(self, dim=1):
        self.dim = dim

    def __call__(self, inputs):
        return inputs.sum(self.dim)

In [14]:
class MyResize(Transform):
    def __init__(self, size=(100, 100)):
        self.size = size

    def __call__(self, inputs):
        image = cv2.resize(inputs.numpy(), dsize=self.size, interpolation=cv2.INTER_CUBIC)
        return torch.tensor(image, dtype=torch.float32)  # Convert back to tensor


In [15]:
class Astype(Transform):
    def __init__(self, dtype=torch.float32):  # Use torch data types
        self.dtype = dtype

    def __call__(self, inputs):
        return inputs.to(self.dtype)  # Convert tensor to specified type

In [16]:
train_transforms = Compose([
    LoadImage(image_only=True),
    Resize((-1,1)),
    Astype(),
    SumDimension(2),
    Astype(),
    MyResize(),
    #AddChannel(),    
    ToTensor(),
])

val_transforms = Compose([
    LoadImage(image_only=True),
    Resize((-1,1)),
    Astype(),
    SumDimension(2),
    Astype(),
    MyResize(),
    #AddChannel(),    
    ToTensor(),
])

act = Activations(softmax=True)
to_onehot = AsDiscrete(to_onehot=num_class)


In [17]:
class MedNISTDataset(Dataset):

    def __init__(self, image_files, labels, transforms):
        self.image_files = image_files
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        return self.transforms(self.image_files[index]), self.labels[index]

In [18]:
train_ds = MedNISTDataset(trainX, trainY, train_transforms)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)

val_ds = MedNISTDataset(valX, valY, val_transforms)
val_loader = DataLoader(val_ds, batch_size=64, num_workers=2)

test_ds = MedNISTDataset(testX, testY, val_transforms)
test_loader = DataLoader(test_ds, batch_size=64, num_workers=2)

In [22]:
device = torch.device("cpu")
model = DenseNet121(
    spatial_dims=2,
    in_channels=1,
    out_channels=num_class
).to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 1e-5)
epoch_num = 4
val_interval = 1

In [23]:
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = list()
auc_metric = ROCAUCMetric()
metric_values = list()
for epoch in range(epoch_num):
    print('-' * 10)
    print(f"epoch {epoch + 1}/{epoch_num}")
    model.train()
    epoch_loss = 0
    step = 0
    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        print(f"{step}/{len(train_ds) // train_loader.batch_size}, train_loss: {loss.item():.4f}")
        epoch_len = len(train_ds) // train_loader.batch_size
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            y_pred = torch.tensor([], dtype=torch.float32, device=device)
            y = torch.tensor([], dtype=torch.long, device=device)
            for val_data in val_loader:
                val_images, val_labels = val_data[0].to(device), val_data[1].to(device)
                y_pred = torch.cat([y_pred, model(val_images)], dim=0)
                y = torch.cat([y, val_labels], dim=0)
            y_onehot = [to_onehot(i) for i in y]
            y_pred_act = [act(i) for i in y_pred]
            auc_metric(y_pred_act, y_onehot)
            auc_result = auc_metric.aggregate()
            auc_metric.reset()
            del y_pred_act, y_onehot
            metric_values.append(auc_result)
            acc_value = torch.eq(y_pred.argmax(dim=1), y)
            acc_metric = acc_value.sum().item() / len(acc_value)
            if acc_metric > best_metric:
                best_metric = acc_metric
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), 'best_metric_model.pth')
                print('saved new best metric model')
            print(f"current epoch: {epoch + 1} current AUC: {auc_result:.4f}"
                  f" current accuracy: {acc_metric:.4f} best AUC: {best_metric:.4f}"
                  f" at epoch: {best_metric_epoch}")
print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

----------
epoch 1/4


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/workbench/.local/lib/python3.10/site-packages/monai/transforms/transform.py", line 141, in apply_transform
    return _apply_transform(transform, data, unpack_items, lazy, overrides, log_stats)
  File "/home/workbench/.local/lib/python3.10/site-packages/monai/transforms/transform.py", line 98, in _apply_transform
    return transform(data, lazy=lazy) if isinstance(transform, LazyTrait) else transform(data)
  File "/home/workbench/.local/lib/python3.10/site-packages/monai/transforms/io/array.py", line 290, in __call__
    img = MetaTensor.ensure_torch_and_prune_meta(
  File "/home/workbench/.local/lib/python3.10/site-packages/monai/data/meta_tensor.py", line 554, in ensure_torch_and_prune_meta
    img = convert_to_tensor(im, track_meta=get_track_meta() and meta is not None)  # potentially ascontiguousarray
  File "/home/workbench/.local/lib/python3.10/site-packages